In [ ]:

# MAINTENANCE PRÉDICTIVE - FRAMEWORK TESLA
# Description : Prédiction de durée de vie des moteurs turbofan
# Approche : Analyse multi-dimensionnelle inspirée de Tesla


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
import os
import joblib
import json
from datetime import datetime

warnings.filterwarnings('ignore')

# Configuration
RANDOM_SEED = 42
DATA_DIR = 'data'
MODEL_DIR = 'models'
VIZ_DIR = 'visualizations'

print("Maintenance Prédictive - Framework Tesla")
print("=" * 60)


# INITIALISATION


def create_directories():
    """Crée la structure de dossiers du projet"""
    for directory in [DATA_DIR, MODEL_DIR, VIZ_DIR]:
        os.makedirs(directory, exist_ok=True)
    print("Structure du projet initialisée")


# ÉTAPE 1 : GÉNÉRATION DES DONNÉES


def create_turbofan_data(n_engines=50, save_data=True):
    """
    Génère des données synthétiques de dégradation de moteurs turbofan.
    
    Basé sur le framework Tesla à 4 dimensions :
    - Énergie : température du système
    - Vibration : signature mécanique  
    - Force : pression interne
    - Information : efficacité globale
    """
    print("\nGénération des données moteurs")
    print("-" * 40)
    
    np.random.seed(RANDOM_SEED)
    data = []
    
    for engine_id in range(1, n_engines + 1):
        # Chaque moteur a une durée de vie différente
        max_cycles = np.random.randint(150, 250)
        
        for cycle in range(1, max_cycles + 1):
            # Modélisation de la dégradation progressive
            degradation = (cycle / max_cycles) ** 2
            noise_level = 0.1 + degradation * 0.2
            
            # Les 4 dimensions du framework Tesla
            temperature = 500 + degradation * 200 + np.random.normal(0, noise_level * 20)
            vibration = 0.5 + degradation * 1.5 + np.random.normal(0, noise_level * 0.3)
            pressure = 40 - degradation * 15 + np.random.normal(0, noise_level * 2)
            efficiency = 0.9 - degradation * 0.2 + np.random.normal(0, noise_level * 0.05)
            
            # Durée de vie restante
            remaining_useful_life = max_cycles - cycle
            
            data.append({
                'engine_id': engine_id,
                'cycle': cycle,
                'temperature': max(300, temperature),
                'vibration': max(0.1, vibration),
                'pressure': max(10, pressure),
                'efficiency': max(0.3, min(1.0, efficiency)),
                'remaining_useful_life': remaining_useful_life,
            })
    
    df = pd.DataFrame(data)
    
    print(f"Dataset créé : {len(df):,} échantillons")
    print(f"Nombre de moteurs : {n_engines}")
    print(f"RUL moyen : {df['remaining_useful_life'].mean():.1f} cycles")
    
    if save_data:
        filepath = os.path.join(DATA_DIR, 'turbofan_data.csv')
        df.to_csv(filepath, index=False)
        print(f"Données sauvegardées dans : {filepath}")
    
    return df


# ÉTAPE 2 : EXPLORATION DES DONNÉES


def explore_data(df):
    """
    Analyse exploratoire des données avec focus sur les corrélations
    entre les 4 dimensions et la durée de vie restante
    """
    print("\nExploration des données")
    print("-" * 40)
    
    # Statistiques générales
    print(f"Shape : {df.shape}")
    print(f"Moteurs uniques : {df['engine_id'].nunique()}")
    
    # Analyse RUL
    rul = df['remaining_useful_life']
    print(f"\nStatistiques RUL :")
    print(f"  Moyenne : {rul.mean():.1f} cycles")
    print(f"  Médiane : {rul.median():.1f} cycles")
    print(f"  Min-Max : {rul.min()}-{rul.max()} cycles")
    
    # Corrélations avec RUL
    sensors = ['temperature', 'vibration', 'pressure', 'efficiency']
    correlations = df[sensors + ['remaining_useful_life']].corr()['remaining_useful_life']
    
    print(f"\nCorrélations avec RUL :")
    for sensor in sensors:
        corr = correlations[sensor]
        print(f"  {sensor.capitalize()} : {corr:.3f}")
    
    return correlations


# ÉTAPE 3 : VISUALISATIONS


def create_visualizations(df, save_plots=True):
    """
    Crée et sauvegarde les visualisations principales du projet
    """
    print("\nCréation des visualisations")
    print("-" * 40)
    
    plt.style.use('seaborn-v0_8-darkgrid')
    
    # Figure 1 : Vue d'ensemble 2x2
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle("Analyse Multi-Dimensionnelle des Moteurs Turbofan", fontsize=14)
    
    # Distribution RUL
    axes[0,0].hist(df['remaining_useful_life'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,0].set_title('Distribution de la Durée de Vie Restante')
    axes[0,0].set_xlabel('RUL (cycles)')
    axes[0,0].set_ylabel('Fréquence')
    
    # Évolution température
    sample_engines = df['engine_id'].unique()[:5]
    for engine in sample_engines:
        engine_data = df[df['engine_id'] == engine]
        axes[0,1].plot(engine_data['cycle'], engine_data['temperature'], 
                        alpha=0.7, label=f'Moteur {engine}')
    axes[0,1].set_title('Évolution de la Température')
    axes[0,1].set_xlabel('Cycle')
    axes[0,1].set_ylabel('Température (°C)')
    axes[0,1].legend()
    
    # Scatter température vs RUL
    axes[1,0].scatter(df['temperature'], df['remaining_useful_life'], 
                        alpha=0.5, c=df['cycle'], cmap='viridis', s=20)
    axes[1,0].set_title('Température vs RUL')
    axes[1,0].set_xlabel('Température (°C)')
    axes[1,0].set_ylabel('RUL (cycles)')
    
    # Matrice de corrélation
    sensors = ['temperature', 'vibration', 'pressure', 'efficiency']
    corr_matrix = df[sensors].corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, ax=axes[1,1])
    axes[1,1].set_title('Corrélations entre Capteurs')
    
    plt.tight_layout()
    
    if save_plots:
        plt.savefig(os.path.join(VIZ_DIR, 'analysis_overview.png'), dpi=150, bbox_inches='tight')
        print("Visualisations sauvegardées dans le dossier visualizations/")
    
    plt.show()


# ÉTAPE 4 : ENTRAÎNEMENT DU MODÈLE


def train_model(df, save_model=True):
    """
    Entraîne un modèle Random Forest pour prédire la durée de vie restante
    """
    print("\nEntraînement du modèle")
    print("-" * 40)
    
    # Préparation des données
    features = ['temperature', 'vibration', 'pressure', 'efficiency']
    X = df[features]
    y = df['remaining_useful_life']
    
    # Division train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_SEED
    )
    
    print(f"Données d'entraînement : {len(X_train)} échantillons")
    print(f"Données de test : {len(X_test)} échantillons")
    
    # Configuration et entraînement du modèle
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        random_state=RANDOM_SEED,
        n_jobs=-1
    )
    
    print("Entraînement en cours...")
    model.fit(X_train, y_train)
    
    # Évaluation
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    print(f"\nPerformances du modèle :")
    print(f"  MAE (train) : {train_mae:.2f} cycles")
    print(f"  MAE (test)  : {test_mae:.2f} cycles")
    print(f"  R² (train)  : {train_r2:.3f}")
    print(f"  R² (test)   : {test_r2:.3f}")
    
    # Vérification overfitting
    if test_mae > train_mae * 1.5:
        print("\nAttention : Possible surapprentissage détecté")
    else:
        print("\nPas de surapprentissage significatif")
    
    # Importance des features
    importance_df = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nImportance des variables :")
    for _, row in importance_df.iterrows():
        print(f"  {row['feature']} : {row['importance']:.3f}")
    
    if save_model:
        model_path = os.path.join(MODEL_DIR, 'random_forest_model.pkl')
        joblib.dump(model, model_path)
        print(f"\nModèle sauvegardé dans {model_path}")
        
        # Sauvegarde des métriques
        metrics = {
            'train_mae': float(train_mae),
            'test_mae': float(test_mae),
            'train_r2': float(train_r2),
            'test_r2': float(test_r2),
            'feature_importance': importance_df.to_dict('records'),
            'training_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        metrics_path = os.path.join(MODEL_DIR, 'model_metrics.json')
        with open(metrics_path, 'w') as f:
            json.dump(metrics, f, indent=2)
    
    return model, X_test, y_test, y_test_pred, test_mae, test_r2


# ÉTAPE 5 : ANALYSE DES RÉSULTATS


def analyze_predictions(y_test, y_pred, save_plots=True):
    """
    Analyse détaillée des prédictions du modèle
    """
    print("\nAnalyse des prédictions")
    print("-" * 40)
    
    # Calcul des erreurs
    errors = y_test - y_pred
    abs_errors = np.abs(errors)
    
    print(f"Erreur moyenne : {np.mean(errors):.2f} cycles")
    print(f"Erreur médiane : {np.median(errors):.2f} cycles")
    print(f"Erreur maximale : {np.max(abs_errors):.2f} cycles")
    
    # Visualisation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Prédictions vs Réalité
    ax1.scatter(y_test, y_pred, alpha=0.6, color='blue', edgecolor='black', linewidth=0.5)
    ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', lw=2, label='Prédiction parfaite')
    ax1.set_xlabel('RUL Réel (cycles)')
    ax1.set_ylabel('RUL Prédit (cycles)')
    ax1.set_title('Prédictions vs Réalité')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Distribution des erreurs
    ax2.hist(errors, bins=30, alpha=0.7, color='green', edgecolor='black')
    ax2.axvline(0, color='red', linestyle='dashed', linewidth=2)
    ax2.set_xlabel('Erreur de prédiction (cycles)')
    ax2.set_ylabel('Fréquence')
    ax2.set_title('Distribution des Erreurs')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_plots:
        plt.savefig(os.path.join(VIZ_DIR, 'prediction_analysis.png'), dpi=150, bbox_inches='tight')
    
    plt.show()

# FONCTION PRINCIPALE


def main():
    """
    Pipeline complet d'analyse prédictive
    """
    print("\n" + "="*60)
    print("PROJET MAINTENANCE PRÉDICTIVE - FRAMEWORK TESLA")
    print("="*60)
    print("Objectif : Prédire la durée de vie des moteurs turbofan")
    print("Approche : Analyse multi-dimensionnelle (Tesla)")
    print("="*60)
    
    try:
        # Initialisation
        create_directories()
        
        # 1. Génération des données
        df = create_turbofan_data(n_engines=50)
        
        # 2. Exploration
        correlations = explore_data(df)
        
        # 3. Visualisations
        create_visualizations(df)
        
        # 4. Machine Learning
        model, X_test, y_test, y_pred, test_mae, test_r2 = train_model(df)
        
        # 5. Analyse des résultats
        analyze_predictions(y_test, y_pred)
        
        # Résumé final
        print("\n" + "="*60)
        print("RÉSUMÉ DU PROJET")
        print("="*60)
        print(f"Modèle entraîné avec succès")
        print(f"Performance : MAE = {test_mae:.1f} cycles, R² = {test_r2:.3f}")
        print(f"Le modèle peut prédire les pannes environ {test_mae:.0f} cycles à l'avance")
        print("\nFichiers générés :")
        print("  - data/turbofan_data.csv")
        print("  - models/random_forest_model.pkl")
        print("  - visualizations/*.png")
        
        return df, model
        
    except Exception as e:
        print(f"Erreur lors de l'exécution : {e}")
        return None, None


# EXÉCUTION


if __name__ == "__main__":
    data, model = main()
    
    print("\nProchaines étapes:")
    print(" Tester avec des données réelles (NASA C-MAPSS)")
